In [34]:
from herbie import Herbie, FastHerbie
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import shapely.geometry as sgeom
import cartopy.feature
import cartopy.feature as cfeature
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
import numpy as np
import pandas as pd
import glob
import matplotlib.cm as cm
import xarray as xr
from datetime import datetime, timedelta
import time
import os
import warnings
from dask import delayed, compute
from dask.distributed import Client, LocalCluster

warnings.filterwarnings("ignore", message="Will not remove GRIB file because it previously existed.")
warnings.filterwarnings("ignore", category=FutureWarning, message=".*decode_timedelta.*")

"""
HRRR model documentation:
https://rapidrefresh.noaa.gov/hrrr/
Documentation for the HRRR data download wrapper:
https://herbie.readthedocs.io/en/stable/
Variables:
https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf00.grib2.shtml
"""

#%% Define

# Folder to save datasets
save_folder = "/kfs2/projects/sfcwinds/HRRR_pointwise_new"

G3P3 = (-106.510100,  34.962400, "G3P3")

# date range with one-hour frequency, reversed (starts from end)
date_range = pd.date_range(datetime(2025, 3, 21), 
                           datetime(2025, 6, 3), 
                           freq="h").tolist()[::-1]

# Forecast hour(s)
fxx =  [0,1,2,3,4,8,12,16,17,18]




In [39]:
fxx = list(range(0,19))
H_subh = FastHerbie(['2025-05-19T23:00:00'], model='hrrr', product='subh', verbose=False, fxx = fxx)

t = H_subh.xarray('GUST', remove_grib=True)


# Add the valid_time coordinate
#hrrr = hrrr.assign_coords(valid_time=hrrr.time + hrrr.step)

Could not find 1/19 GRIB files.


In [38]:
t

<xarray.Dataset>
Dimensions:              (step: 73, y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2025-05-19T23:00:00
  * step                 (step) timedelta64[ns] 00:00:00 00:15:00 ... 18:00:00
    surface              float64 0.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           (step) datetime64[ns] 2025-05-19T23:00:00 ... 2025-0...
    gribfile_projection  object None
Dimensions without coordinates: y, x
Data variables:
    gust                 (y, x, step) float32 7.114 7.181 7.19 ... 14.35 14.43
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 subh
    description:             High-Resolution Rapid Refresh - CONUS
    search:                  GUST

In [24]:
date_range[0]

Timestamp('2025-06-03 00:00:00')

In [32]:
synoptic_hours = [0, 6, 12, 18]


# Determine forecast range
forecast_range = 49 if date_range[1].hour in synoptic_hours else 19

# Create list of forecast hours
fxx = list(range(forecast_range))

In [33]:
fxx

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]